In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=5b8a55efd1c8577b3eb834fcba69ce7dacd1f18762bca75a8ae6d3f24e2e50f0
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
lti_spark=SparkSession.builder.appName("LTI").getOrCreate()

In [5]:
%%writefile movies.csv

movieId,movieName,year
1,Parmanu,2018
2,URI,2019
3,rrr,2022
4,Bahubali,2015
5,tanhaji,2020

Writing movies.csv


In [6]:
%%writefile review.csv
movieId,movieInfo,reviews
1,,435
2,,492
3,,444
4,,880
5,,722
6,,888
7,,220
8,,350
9,,619
10,,226

Writing review.csv


In [8]:
rdf=lti_spark.read.csv("/content/review.csv",header=True)
mdf=lti_spark.read.csv("/content/movies.csv",header=True)

 Clean reviews dataframe on the basis of movies data available. Hint - try left semi join, inner join, right join for joins
2.  Fill movieInfo Column by concating movieName & year column (output should be dataframe).  Hint - try expr method
3. Find movie with max reviews (output should be string)
4. Find total reviews (output should be int)

In [9]:
rdf.printSchema()

root
 |-- movieId: string (nullable = true)
 |-- movieInfo: string (nullable = true)
 |-- reviews: string (nullable = true)



In [10]:
rdf.show()

+-------+---------+-------+
|movieId|movieInfo|reviews|
+-------+---------+-------+
|      1|     NULL|    435|
|      2|     NULL|    492|
|      3|     NULL|    444|
|      4|     NULL|    880|
|      5|     NULL|    722|
|      6|     NULL|    888|
|      7|     NULL|    220|
|      8|     NULL|    350|
|      9|     NULL|    619|
|     10|     NULL|    226|
+-------+---------+-------+



In [11]:
rdf1=rdf.na.fill(subset=["movieInfo"])
rdf1.show()

+-------+---------+-------+
|movieId|movieInfo|reviews|
+-------+---------+-------+
+-------+---------+-------+



In [13]:
#Clean reviews dataframe on the basis of movies data available. Hint - try left semi join, inner join, right join for joins
rdf.join(mdf,rdf.movieId==mdf.movieId,"leftsemi").show()

+-------+---------+-------+
|movieId|movieInfo|reviews|
+-------+---------+-------+
|      1|     NULL|    435|
|      2|     NULL|    492|
|      3|     NULL|    444|
|      4|     NULL|    880|
|      5|     NULL|    722|
+-------+---------+-------+



In [18]:
jdf=rdf.join(mdf,"movieId","right")
jdf.show()

+-------+---------+-------+---------+----+
|movieId|movieInfo|reviews|movieName|year|
+-------+---------+-------+---------+----+
|      1|     NULL|    435|  Parmanu|2018|
|      2|     NULL|    492|      URI|2019|
|      3|     NULL|    444|      rrr|2022|
|      4|     NULL|    880| Bahubali|2015|
|      5|     NULL|    722|  tanhaji|2020|
+-------+---------+-------+---------+----+



In [20]:
jdf=jdf.withColumn("movieInfo",concat_ws("-","movieName","year"))
jdf.show()

+-------+-------------+-------+---------+----+
|movieId|    movieInfo|reviews|movieName|year|
+-------+-------------+-------+---------+----+
|      1| Parmanu-2018|    435|  Parmanu|2018|
|      2|     URI-2019|    492|      URI|2019|
|      3|     rrr-2022|    444|      rrr|2022|
|      4|Bahubali-2015|    880| Bahubali|2015|
|      5| tanhaji-2020|    722|  tanhaji|2020|
+-------+-------------+-------+---------+----+



In [24]:
jdf2 = rdf.join(mdf,"movieId","right")
jdf2.show()

+-------+---------+-------+---------+----+
|movieId|movieInfo|reviews|movieName|year|
+-------+---------+-------+---------+----+
|      1|     NULL|    435|  Parmanu|2018|
|      2|     NULL|    492|      URI|2019|
|      3|     NULL|    444|      rrr|2022|
|      4|     NULL|    880| Bahubali|2015|
|      5|     NULL|    722|  tanhaji|2020|
+-------+---------+-------+---------+----+



In [34]:
jdf2.withColumn("movieInfo",expr("movieName||'-'||year")).show()

+-------+-------------+-------+---------+----+
|movieId|    movieInfo|reviews|movieName|year|
+-------+-------------+-------+---------+----+
|      1| Parmanu-2018|    435|  Parmanu|2018|
|      2|     URI-2019|    492|      URI|2019|
|      3|     rrr-2022|    444|      rrr|2022|
|      4|Bahubali-2015|    880| Bahubali|2015|
|      5| tanhaji-2020|    722|  tanhaji|2020|
+-------+-------------+-------+---------+----+



In [57]:
# c=jdf2.select(max("reviews")).collect()[0][0]
# print(c)
jdf2.groupby("movieName").agg(max("reviews")).first()[0]

'Bahubali'

In [49]:
#Find total reviews (output should be int)
jdf2.select(count("reviews")).collect()[0][0]

5

In [58]:
%%writefile movie1.csv
movieId,movieName,year,fansReview,criticsReview
1,Parmanu,2018,200,100
2,URI,2019,300,200
3,rrr,2022,300,200
4,Bahubali,2015,500,400
5,tanhaji,2020,600,500

Writing movie1.csv


In [59]:
%%writefile genre.csv
movieId,genre
1,tech
2,history
2,action
4,action
3,action
1,science
4,fiction
5,history
5,action
4,mass

Writing genre.csv


In [60]:
df=lti_spark.read.csv("/content/movie1.csv",header=True)
gdf=lti_spark.read.csv("/content/genre.csv",header=True)

In [67]:
#Find top movies by fansReview to criticsReview ratio. Hint - Add a column for same
df=df.withColumn("ratio",col("fansReview")/col("criticsReview"))
df.sort("ratio",ascending=False).collect()[0][1]

'Parmanu'

In [68]:
#Order moviesDF by year to get latest movie
df.sort("year",ascending=False).show(1)

+-------+---------+----+----------+-------------+-----+
|movieId|movieName|year|fansReview|criticsReview|ratio|
+-------+---------+----+----------+-------------+-----+
|      3|      rrr|2022|       300|          200|  1.5|
+-------+---------+----+----------+-------------+-----+
only showing top 1 row



In [75]:
#Join dataframes on movieID & find most popular genre
df.join(gdf,"movieId").groupby("genre").count().first()[0]

'action'